In [9]:
using Pkg
Pkg.add("DifferentialEquations")
Pkg.add("Plots")
Pkg.add("SpecialFunctions")
Pkg.add("LegendrePolynomials")
Pkg.add("SphericalFunctions")



   Resolving package versions...


  No Changes to `C:\Users\Seiven\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\Seiven\.julia\environments\v1.8\Manifest.toml`


   Resolving package versions...


  No Changes to `C:\Users\Seiven\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\Seiven\.julia\environments\v1.8\Manifest.toml`

   Resolving package versions...


...

  ✓ Hwloc_jll
  ✓ GenericLinearAlgebra


  ✓ Hwloc


  ✓ Quaternionic


  ✓ DoubleFloats


  ✓ SphericalFunctions
  7 dependencies successfully precompiled in 12 seconds. 407 already precompiled.


In [10]:
using DifferentialEquations
using Plots
using SpecialFunctions
using LegendrePolynomials
using SphericalFunctions

# Parameters
l_max = 4
sigma = 0.2
t_max = 10.0
n_points = 100

# Define theta and phi arrays
theta = range(0, stop=π, length=n_points)
phi = range(0, stop=2π, length=n_points)
Theta, Phi = [t for p in phi, t in theta], [p for p in phi, t in theta]

# Initial condition function
function initial_condition(θ, σ)
    return exp(-θ^2 / (2 * σ^2))
end

# Associated Legendre function
function assoc_legendre(l, m, x)
    coeff = ((-1)^m) * sqrt((2l + 1) / (4π) * factorial(l - m) / factorial(l + m))
    return coeff * Plm(x, l, abs(m))
end

function spherical_harmonics(m, l, ϕ, θ)
    # SphericalHarmonicY takes arguments (l, m, θ, ϕ)
    # Note: SphericalFunctions.jl uses physicists' convention by default
    return SphericalHarmonicY(l, m, θ, ϕ)
end

# Project initial condition onto spherical harmonics basis
function project_initial_condition(l, m, σ, θ_vals, ϕ_vals)
    integral = 0.0
    for θ_val in θ_vals
        for ϕ_val in ϕ_vals
            Y_lm = conj(spherical_harmonics(m, l, ϕ_val, θ_val))
            integral += initial_condition(θ_val, σ) * Y_lm * sin(θ_val)
        end
    end
    Δθ = θ_vals[2] - θ_vals[1]
    Δϕ = ϕ_vals[2] - ϕ_vals[1]
    return integral * Δθ * Δϕ
end

# ODE system for c_lm(t) and psi_lm(t)
function ode_system(du, u, p, t)
    l_max = p
    for l in 0:l_max
        for m in -l:l
            idx = l^2 + l + m + 1
            du[idx] = u[idx+1]
            du[idx+1] = -l * (l + 1) * u[idx]
        end
    end
end

# Solve the ODE system
function solve_ode(l_max, σ, t_max)
    n_coeffs = (l_max + 1)^2
    y0 = zeros(2 * n_coeffs)  # Expected to be real
    for l in 0:l_max
        for m in -l:l
            idx = l^2 + l + m + 1
            y0[idx] = real(project_initial_condition(l, m, σ, theta, phi))
        end
    end
    tspan = (0.0, t_max)
    prob = ODEProblem(ode_system, y0, tspan, l_max)
    sol = solve(prob, Tsit5(), reltol=1e-8)
    return sol
end


# Visualization of the solution at a given time t
function visualize_solution(sol, t, l_max)
    Z = zeros(length(theta), length(phi))  # Ensure Z is for real numbers
    for l in 0:l_max
        for m in -l:l
            idx = l^2 + l + m + 1
            for i in 1:length(theta)
                for j in 1:length(phi)
                    # Explicitly take the real part of the spherical harmonics calculation
                    Z[i, j] += real(spherical_harmonics(m, l, phi[j], theta[i]) * sol(t)[idx])
                end
            end
        end
    end
    contourf(phi, theta, Z, title="Solution at t=$t", xlabel="Phi", ylabel="Theta", color=:viridis)
end


# Main execution
l_max_values = [4, 8, 12]  # Different l_max values for comparison
for l_max_val in l_max_values
    sol = solve_ode(l_max_val, sigma, t_max)
    # Visualize at different time steps
    for t in range(0, stop=t_max, length=5)
        visualize_solution(sol, t, l_max_val)
    end
end


UndefVarError: UndefVarError: SphericalHarmonicY not defined